In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [209]:
# Loading the fake and real datasets
data_fake = pd.read_csv("/Users/georgychomakhashvili/PycharmProjects/FakeNews/Fake.csv")
data_true = pd.read_csv("/Users/georgychomakhashvili/PycharmProjects/FakeNews/True.csv")

In [3]:
# Checking the shape of the datasets
data_fake.head()

In [4]:
data_true.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [160]:
# Assigning a class label to the datasets
data_fake["class"] = 0
data_true["class"] = 1

In [161]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [162]:
# Creating a manual testing dataset from the tail of the original datasets
data_fake_manual_testing = data_fake.tail(10)
for i in range(data_fake.shape[0]-1, data_fake.shape[0]-11, -1):
    data_fake.drop([i], axis = 0, inplace = True)
    
data_true_manual_testing = data_true.tail(10)
for i in range(data_true.shape[0]-1, data_true.shape[0]-11, -1):
    data_true.drop([i], axis = 0, inplace = True)

In [163]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [164]:
data_fake_manual_testing["class"] = 0
data_true_manual_testing["class"] = 1

/var/folders/yv/cmb94hwd1zsb37rvdwxyl34r0000gn/T/ipykernel_1122/139271216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing["class"] = 0
/var/folders/yv/cmb94hwd1zsb37rvdwxyl34r0000gn/T/ipykernel_1122/139271216.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing["class"] = 1


In [165]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [166]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [167]:
# Merging the fake and real datasets
data_merge = pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [168]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [169]:
data = data_merge.drop(["title", "subject", "date"], axis = 1)

In [170]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [171]:
data = data.sample(frac = 1)

In [172]:
data.head()

,text,class
4267,"With 28 days left to go until election day, a ...",0
19446,,0
11026,FORMER WHITE HOUSE ADVISOR STEPHANIE CUTTER to...,0
11815,The Democratic Party held a forum for race obs...,0
19124,Description advertising racist wear from Faceb...,0


In [173]:
# Resetting the index of the dataset
data.reset_index(inplace = True)
data.drop(["index"], axis = 1, inplace = True)

In [174]:
data.columns

Index(['text', 'class'], dtype='object')

In [175]:
data.head()

,text,class
0,"With 28 days left to go until election day, a ...",0
1,,0
2,FORMER WHITE HOUSE ADVISOR STEPHANIE CUTTER to...,0
3,The Democratic Party held a forum for race obs...,0
4,Description advertising racist wear from Faceb...,0


In [176]:
# A function to preprocess the text
def wordopt(text):
    text = text.lower()
    text = re.sub("\[.*?\]", "", text)
    text = re.sub("\\W"," ", text)
    text = re.sub("https?://\S+|www\.\S+", "", text)
    text = re.sub("<.*?>+", "", text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re. sub("\n", "", text)
    text = re.sub("\w*\d\w*", "", text)
    return text

In [177]:
# Applying the preprocessing function to the 'text' column
data["text"] = data["text"].apply(wordopt)

In [178]:
# Splitting the data into features and target variable
x = data["text"]
y = data["class"]

In [179]:
# Splitting the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25)

In [180]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()

# Fitting the vectorizer to the training data and transforming the training data into a matrix of TF-IDF features
xv_train = vectorization.fit_transform(x_train)

# Transforming the test data into a matrix of TF-IDF features using the vectorizer fitted to the training data
xv_test = vectorization.transform(x_test)

In [181]:
from sklearn.linear_model import LogisticRegression

# Training the Logistic Regression model
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [182]:
# Predicting the test set results and calculating the accuracy
pred_lr = LR.predict(xv_test)

In [183]:
LR.score(xv_test, y_test)

0.9868092691622103

In [184]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5939
           1       0.98      0.99      0.99      5281

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220


In [185]:
from sklearn.tree import DecisionTreeClassifier

# Training the Decision Tree model
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [186]:
# Predicting the test set results and calculating the accuracy
pred_dt = DT.predict(xv_test)

In [187]:
DT.score(xv_test, y_test)

0.9951871657754011

In [188]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5939
           1       1.00      0.99      0.99      5281

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220


In [189]:
from sklearn.ensemble import GradientBoostingClassifier

# Training the Gradient Boosting model
GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [190]:
# Predicting the test set results and calculating the accuracy
pred_gb = GB.predict(xv_test)

In [191]:
GB.score(xv_test, y_test)

0.9958110516934047

In [192]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5939
           1       0.99      1.00      1.00      5281

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220


In [193]:
from sklearn.ensemble import RandomForestClassifier

# Training the Random Forest model
RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [194]:
# Predicting the test set results and calculating the accuracy
pred_rf = RF.predict(xv_test)

In [195]:
RF.score(xv_test, y_test)

0.989572192513369

In [196]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5939
           1       0.99      0.99      0.99      5281

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220


In [197]:
# A function to output the label
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not a Fake News"
    
# A function for manual testing    
def manual_testing(news):
    testing_news = {"text" : [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)
    
    return print("\n\nLR Prediction: {} \nDT Prediciton: {} \nGBC Prediciton: {} \nRFC Prediciton: {}".format(output_lable(pred_LR[0]), output_lable(pred_DT[0]),output_lable(pred_GB[0]),output_lable(pred_RF[0])))

In [198]:
news = str(input())
manual_testing(news)



LR Prediction: Fake News 
DT Prediciton: Fake News 
GBC Prediciton: Fake News 
RFC Prediciton: Fake News
